import some package : 
- panda will help you manipulate tables
- numpy is helpful for computations
- matplotlib makes pretty graphs
- statsmodels will help you compute some interesting statistical decompositions 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import  STL

we load the johnson and johnson dataset, it shows the stock price per date good practice suggests you plot the head so that you see the dataset structure

In [ ]:
df = pd.read_csv('../data/jj.csv')
df.head()

In [ ]:
df.tail()

let's have a new column with the year, then describe your dataset to grasp distribution 


In [ ]:
df['year'] = pd.DatetimeIndex(df['date']).year
df.describe()

always good practice to visualize also 
fig.autofmt_xdate() put x labels diagonally for better reading 
plt.tight_layout() removes space around the figure

In [ ]:
fig, ax = plt.subplots()

ax.plot(df.date, df.data)
ax.set_xlabel('Date')
ax.set_ylabel('Earnings per share (USD)')

plt.xticks(np.arange(0, 85, 8), [1960, 1962, 1964, 1966, 1968, 1970, 1972, 1974, 1976, 1978, 1980])

fig.autofmt_xdate()
plt.tight_layout()


STL fit method estimates season, trend and residuals components.

In [ ]:
advanced_decomposition = STL(df.data, period=4).fit()

In [ ]:
# noinspection PyTypeChecker
fig, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=4, ncols=1, sharex=True)

ax1.plot(advanced_decomposition.observed)
ax1.set_ylabel('Observed')

ax2.plot(advanced_decomposition.trend)
ax2.set_ylabel('Trend')

ax3.plot(advanced_decomposition.seasonal)
ax3.set_ylabel('Seasonal')

ax4.plot(advanced_decomposition.resid)
ax4.set_ylabel('Residuals')


plt.xticks(np.arange(0, 84, 8), [1960, 1962, 1964, 1966, 1968, 1970, 1972, 1974, 1976, 1978, 1980])

fig.autofmt_xdate()
plt.tight_layout()


In [ ]:
fig, ax = plt.subplots()

ax.plot(df.date, df.data)
ax.plot(advanced_decomposition.trend, color='lightgrey', linestyle='--', label='Trend')
ax.set_xlabel('Date')
ax.set_ylabel('Earnings per share (USD)')

plt.xticks(np.arange(0, 85, 8), [1960, 1962, 1964, 1966, 1968, 1970, 1972, 1974, 1976, 1978, 1980])

fig.autofmt_xdate()
plt.tight_layout()


Since seasonality seems operating per year lets see if given every year except the last one (four data points, since a year has four quarters), we are able to predict the value of the missing year. 
The idea is to operate incrementally: make a simple prediction model as baseline and every iteration we have a new model and evaluate if the new model is better, if yes, replace baseline

In [ ]:
train = df[:-4]
test = df[-4:]

We need to define an evaluation strategy. A simple standard one would be mape: 
for each data point we take the difference between predicted and observed value, then the score is the mean of the deviations 

In [ ]:
def mape(y_true, y_pred):
    val = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('average error is',val)
    return val

and let's add a function to easily show results 

In [ ]:
def print_result_graph(column):
    _fig, _ax = plt.subplots()

    _ax.plot(train['date'], train['data'], 'g-.', label='Train')
    _ax.plot(test['date'], test['data'], 'b-', label='Test')
    _ax.plot(test['date'], test[column], 'r--', label='Predicted')
    _ax.set_xlabel('Date')
    _ax.set_ylabel('Earnings per share (USD)')
    _ax.axvspan(80, 83, color='#808080', alpha=0.2)
    _ax.legend(loc=2)

    plt.xticks(np.arange(0, 85, 8), [1960, 1962, 1964, 1966, 1968, 1970, 1972, 1974, 1976, 1978, 1980])

    _fig.autofmt_xdate()
    plt.tight_layout()

The first strategy is to use the historical mean as the first baseline

In [ ]:
historical_mean = np.mean(train['data'])
test.loc[:, 'pred_histo_mean'] = historical_mean
mape_hist_mean = mape(test['data'], test['pred_histo_mean'])
print_result_graph("pred_histo_mean")


the score is awful, but that was to be expected since the trend is moving,
the diagram is pretty clear on why 

let's implement a new method: mean only on last year 

In [ ]:
last_year_mean = np.mean(train['data'][-4:])
test.loc[:, 'pred__last_yr_mean'] = last_year_mean
mape_last_year_mean = mape(test['data'], test['pred__last_yr_mean'])
print_result_graph('pred__last_yr_mean')


we can also try to predict the last known value 

In [ ]:
last = train['data'].iloc[-1]
test.loc[:, 'pred_last'] = last
mape_last = mape(test['data'], test['pred_last'])
print_result_graph("pred_last")

error is worse than last year's average 
lets use the seasonality naively: take the value of the same quarter year earlier  

In [ ]:
test.loc[:, 'pred_last_season'] = train['data'][-4:].values
mape_naive_seasonal = mape(test['data'], test['pred_last_season'])
print_result_graph('pred_last_season')

now let's wrap up by comparing everything 

In [ ]:
fig, ax = plt.subplots()

x = ['hist_mean', 'last_year_mean', 'last', 'naive_seasonal']
y = [70.00, 15.60, 30.46, 11.56]

ax.bar(x, y, width=0.4)
ax.set_xlabel('Baselines')
ax.set_ylabel('MAPE (%)')
ax.set_ylim(0, 75)

for index, value in enumerate(y):
    plt.text(x=index, y=value + 1, s=str(value), ha='center')

plt.tight_layout()